In [0]:
%pip install sidrapy

In [0]:
%restart_python

In [0]:
import sidrapy
import pandas as pd

In [0]:
print ("Oi pai. Td bem?")

In [0]:
spark.sql("CREATE SCHEMA IF NOT EXISTS workspace.ibge_bronze")
spark.sql("CREATE SCHEMA IF NOT EXISTS workspace.ibge_silver")
spark.sql("CREATE SCHEMA IF NOT EXISTS workspace.ibge_gold")

spark.sql("USE CATALOG workspace")
print("Schemas criados!")

In [0]:
spark.sql("""
CREATE TABLE IF NOT EXISTS workspace.ibge_bronze.populacao (
    municipio_id INT,
    municipio_nome STRING,
    ano INT,
    pop_valor DOUBLE,
    raw_payload STRING,
    ingest_ts TIMESTAMP,
    source STRING
)
""")


In [0]:
spark.sql("""
CREATE TABLE IF NOT EXISTS workspace.ibge_bronze.pib (
    municipio_id INT,
    municipio_nome STRING,
    ano INT,
    pib_valor DOUBLE,
    unidade STRING,
    raw_payload STRING,
    ingest_ts TIMESTAMP,
    source STRING
)
""")


In [0]:
# Criar tabela SILVER População (tratada)
spark.sql("""
CREATE TABLE IF NOT EXISTS workspace.ibge_silver.populacao AS
SELECT
    municipio_id,
    TRIM(LOWER(municipio_nome)) AS municipio_nome,
    ano,
    CAST(pop_valor AS DOUBLE) AS pop_valor,
    ingest_ts
FROM workspace.ibge_bronze.populacao
WHERE pop_valor IS NOT NULL
""")

print("Tabela workspace.ibge_silver.populacao criada!")
spark.table("workspace.ibge_silver.populacao").show(5)


In [0]:
from pyspark.sql.functions import when, col

# Criar tabela base silver com limpeza
df_silver_pib = spark.sql("""
SELECT
    municipio_id,
    TRIM(LOWER(municipio_nome)) AS municipio_nome,
    ano,
    CAST(pib_valor AS DOUBLE) AS pib_valor,
    unidade,
    ingest_ts
FROM workspace.ibge_bronze.pib
WHERE pib_valor IS NOT NULL
""")

# Normalizar unidade (exemplo: "Mil Reais")
df_silver_pib = df_silver_pib.withColumn(
    "pib_valor",
    when(col("unidade").like("%mil%"), col("pib_valor") * 1000)
    .otherwise(col("pib_valor"))
)

# Salvar como Silver
df_silver_pib.write.mode("overwrite").saveAsTable("workspace.ibge_silver.pib")

print("Tabela workspace.ibge_silver.pib criada!")
spark.table("workspace.ibge_silver.pib").show(5)


In [0]:
# Criar tabela GOLD com indicadores
spark.sql("""
CREATE TABLE IF NOT EXISTS workspace.ibge_gold.pib_pop AS
SELECT
    p.municipio_id,
    p.municipio_nome,
    p.ano,
    p.pop_valor,
    i.pib_valor,
    i.unidade,
    (i.pib_valor / p.pop_valor) AS pib_per_capita
FROM workspace.ibge_silver.populacao p
LEFT JOIN workspace.ibge_silver.pib i
    ON p.municipio_id = i.municipio_id
    AND p.ano = i.ano
""")

print("Tabela workspace.ibge_gold.pib_pop criada!")
spark.table("workspace.ibge_gold.pib_pop").show(10)
